# Assignment 2 of S Deepak Narayanan, 16110142

## Question 2. follows below 
### Question 1. is just to download the dataset, which has already been done

### Tokenization using Tokenizer

In [1]:
from nltk.tokenize import sent_tokenize
file = open('alice.txt').read()
data_temp = sent_tokenize(file)

### Additional Dataset cleaning 

In [2]:
#data_temp = data
for dt in range(len(data_temp)):
    data_temp[dt] = data_temp[dt].replace('\n', "")
    data_temp[dt] = data_temp[dt].replace('\n', "")
    data_temp[dt] = data_temp[dt].replace('\ '," ")
    data_temp[dt] = data_temp[dt].replace('"',' ' )
    data_temp[dt] = data_temp[dt].replace("  "," ")
    data_temp[dt] = data_temp[dt].replace("."," ")
data_final = []
for data in data_temp:
    data = '<s> '+data
    data = data + ' </s>'
    data_final.append(data)
# Final Data is Data Final itself
del data_temp

### Splitting the data into 80:20

In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_final, test_size = 0.2, random_state= 10)
del train_data

## Question 3. follows below

### Unigram Model and MLE for Unigram

In [4]:
unigram_count = {}
for data in data_final:
    arr = data.split()
    for elem in arr:
        if elem in unigram_count:
            unigram_count[elem]+=1
        else:
            unigram_count[elem]=1
## Unigram Count is a dictionary that contains all the Unigrams with their number of occurrences in the corpus.
total_count = sum(unigram_count.values())
mle_unigram = {}
for elem in unigram_count:
    mle_unigram[elem] = unigram_count[elem]/total_count

### Bigram Model and MLE for Bigram

In [5]:
# Bigram Count contains all the bigrams with their number of occurrences in the corpus. 
bigram_count = {}
for data in data_final:
    tmp_arr = data.split()
    for i in range(len(tmp_arr)):
        try:
            if (tmp_arr[i],tmp_arr[i+1]) in bigram_count:
                bigram_count[(tmp_arr[i],tmp_arr[i+1])]+=1
            if (tmp_arr[i], tmp_arr[i+1]) not in bigram_count:
                 bigram_count[(tmp_arr[i],tmp_arr[i+1])]=1
        except:
            continue
mle_bigram = {}
for i in bigram_count:
    mle_bigram[i] = bigram_count[i]/unigram_count[i[0]]

### Trigram Count and MLE for Trigrams

In [6]:
trigram_count = {}
for data in data_final:
    tmp_arr = data.split()
    for i in range(len(tmp_arr)):
        try:
            if (tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2]) in trigram_count:
                trigram_count[(tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2])]+=1
            if (tmp_arr[i], tmp_arr[i+1],tmp_arr[i+2]) not in trigram_count:
                 trigram_count[(tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2])]=1
        except:
            continue
mle_trigram = {}
for i in trigram_count:
    mle_trigram[i] = trigram_count[i]/bigram_count[(i[0],i[1])]

### Quadgram Count and MLE for Quadgram

In [7]:
quadgram_count = {}
for data in data_final:
    tmp_arr = data.split()
    for i in range(len(tmp_arr)):
        try:
            if (tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2], tmp_arr[i+3]) in quadgram_count:
                quadgram_count[(tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2], tmp_arr[i+3])]+=1
            if (tmp_arr[i], tmp_arr[i+1],tmp_arr[i+2], tmp_arr[i+3]) not in quadgram_count:
                 quadgram_count[(tmp_arr[i],tmp_arr[i+1],tmp_arr[i+2], tmp_arr[i+3])]=1
        except:
            continue
mle_quadgram = {}
for i in quadgram_count:
    mle_quadgram[i] = quadgram_count[i]/trigram_count[(i[0],i[1], i[2])]

### The below unic, bigc, tric, quadc, contains the number of unigrams, bigrams, trigrams and quadgrams in the corpus. Using the entire corpus in this case. The number of possible ones is also covered in here.

In [8]:
unic = len(unigram_count)
bigc = len(bigram_count)
tric = len(trigram_count)
quadc = len(quadgram_count)

possible= {i:0 for i in range(1,5)}
possible[1] = unic
possible[2] = unic**2
possible[3] = unic**3
possible[4] = unic**4

present = {1:unic, 2:bigc, 3:tric, 4:quadc}
print("The total number of n-grams possible and those present are respectively")
for i in present:
    print(str(i) + '-gram '+str(possible[i])+','+str(present[i]))

The total number of n-grams possible and those present are respectively
1-gram 5949,5949
2-gram 35390601,17565
3-gram 210538685349,22750
4-gram 1252494639141201,23287


## Q4. (b) Sentence Probabilities
### I am randomly taking 10 sentences from the corpus and displaying their probability using the unigram, bigram, trigram and the quadgram models

In [9]:
from math import log

def prob_sent(sentence, ngram):
    
    # Probability of Formation of a sentence using a Unigram
    if ngram==1:
        flag = 1
        array = sentence.split()
        product = 0
        for i in array:
            try:
                product+=mle_unigram[i]
            except:
                flag = 0
                break
        if(flag==0):
            return 100
        return product, math.exp(product)

    # Probability of formation of a sentence using a bigram
    if ngram==2:
        flag = 1
        array = sentence.split()
        product = 0
        for i in range(len(array)-1):
            try:
                product+=mle_bigram[(array[i],array[i+1])]
            except:
                flag = 0
                break
        if(flag==0):
            return 100
        return product, math.exp(product)
    
    # Probability of sentence formation using trigrams
    if ngram==3:
        flag = 1
        array = sentence.split()
        product = 1
        for i in range(len(array)-2):
            try:
                product+=log(mle_trigram[(array[i],array[i+1], array[i+2])])
            except:
                flag = 0
                break
        if(flag==0):
            return 100
        return product, math.exp(product)
    
    #Probability of a sentence using quadgram 
    if ngram==4:
        flag = 1
        array = sentence.split()
        product = 0
        for i in range(len(array)-3):
            try:
                product+=log(mle_quadgram[(array[i],array[i+1], array[i+2], array[i+3])])
            except:
                flag = 0
                break
        if(flag==0):
            return 100
        return product, math.exp(product)

### The results are below: If 100, then the probability is 0, else the values are those computed from log probabilities 

In [10]:
import math
import random
print( 'The two tuple below shows probability in log scale and actual probability')
sentence_list_test = []
for i in range(10):
    sentence_list_test.append(random.choice(data_final))
for i in range(10):
    print('The sentence chosen is \n',sentence_list_test[i])
    for j in range(4):
         print('The Probability using '+str(j+1)+' gram model is ', prob_sent(sentence_list_test[i],j+1))
    print()

The two tuple below shows probability in log scale and actual probability
The sentence chosen is 
 <s> ‘I dare say you never even spoke to Time!’‘Perhaps not,’ Alice cautiously replied: ‘but I know I have to beat timewhen I learn music ’‘Ah! </s>
The Probability using 1 gram model is  (0.178200821201941, 1.1950652915866815)
The Probability using 2 gram model is  (6.555769420518711, 703.2900675082301)
The Probability using 3 gram model is  (-12.888017248751021, 2.5281639029567037e-06)
The Probability using 4 gram model is  (-1.3862943611198906, 0.25)

The sentence chosen is 
 <s> ‘He denies it,’ said the King: ‘leave out that part ’‘Well, at any rate, the Dormouse said--’ the Hatter went on, lookinganxiously round to see if he would deny it too: but the Dormouse deniednothing, being fast asleep  </s>
The Probability using 1 gram model is  (0.36722657708100037, 1.4437249970551962)
The Probability using 2 gram model is  (10.756992309815788, 46957.22264767012)
The Probability using 3 gram 

## Q4. (a) Generating Sentences by using MLEs for the N-Gram Model

### The input to this is an integer, that can take values from 1 till 4. 
### Output would be a sentence generated
### I have generated 10 sample sentences per n-gram below

In [11]:
import numpy as np
def generate(ngram):
    deep = -100
    sentence = ['<s>']
    start = '<s>'
    if ngram==1:
        while(start!='</s>'):
            relevant=[]
            for i in unigram_count:
                relevant.append(i)
            prob = []
            for i in relevant:
                prob.append(mle_unigram[i])
            arr = np.random.multinomial(1, prob,size = None)
            max_ = {0:0}
            for i in arr:
                max_[0] = max(max_[0],i)
            for i in range(len(arr)):
                if arr[i] == max_[0]:
                    deep = i
                    break
            start = relevant[deep]
            sentence.append(start)
            relevant=[]
            deep = -100
        return ' '.join(sentence)
    if ngram==2:
        while(start!='</s>'):
            relevant = []
            for i in bigram_count:
                if i[0]==start:
                    relevant.append(i)
            prob = []
            for i in relevant:
                prob.append(mle_bigram[i])
            arr = np.random.multinomial(1, prob,size = None)
            max_ = {0:0}
            for i in arr:
                max_[0] = max(max_[0],i)
            for i in range(len(arr)):
                if arr[i] == max_[0]:
                    deep = i
                    break
            start = relevant[deep][1]
            sentence.append(start)
            relevant=[]
            deep = -100
        return ' '.join(sentence)
    if ngram==3:
        start = '<s>'
        relevant = []
        for i in bigram_count:
                if i[0]==start:
                    relevant.append(i)
        prob = []
        for i in relevant:
            prob.append(mle_bigram[i])
        arr = np.random.multinomial(1, prob,size = None)
        max_ = {0:0}
        for i in arr:
            max_[0] = max(max_[0],i)
        for i in range(len(arr)):
            if arr[i] == max_[0]:
                deep = i
                break
        bigram_2 = relevant[deep][1]
        start_trigram = ('<s>',bigram_2)
        sentence = ['<s>',bigram_2]
        while(1):
            relevant = []
            for i in trigram_count:
                if i[0]==start and i[1] ==bigram_2:
                    relevant.append(i)
            prob = []
            for i in relevant:
                prob.append(mle_trigram[i])
            arr = np.random.multinomial(1, prob,size = None)
            max_ = {0:0}
            for i in arr:
                max_[0] = max(max_[0],i)
            for i in range(len(arr)):
                if arr[i] == max_[0]:
                    deep = i
                    break
            start = relevant[deep][1]
            bigram_2=relevant[deep][2]
            sentence.append(bigram_2)
            relevant=[]
            deep = -100
            if bigram_2=='</s>':
                break
        return ' '.join(sentence)
    if ngram==4:
        start = '<s>'
        relevant = []
        for i in bigram_count:
                if i[0]==start:
                    relevant.append(i)
        prob = []
        for i in relevant:
            prob.append(mle_bigram[i])
        arr = np.random.multinomial(1, prob,size = None)
        max_ = {0:0}
        for i in arr:
            max_[0] = max(max_[0],i)
        for i in range(len(arr)):
            if arr[i] == max_[0]:
                deep = i
                break
        bigram_2 = relevant[deep][1]
        sentence = ['<s>',bigram_2]
        relevant = []
        for i in trigram_count:
            if i[0]==start and i[1] ==bigram_2:
                relevant.append(i)
        prob = []
        for i in relevant:
            prob.append(mle_trigram[i])
        arr = np.random.multinomial(1, prob,size = None)
        max_ = {0:0}
        for i in arr:
            max_[0] = max(max_[0],i)
        for i in range(len(arr)):
            if arr[i] == max_[0]:
                deep = i
                break
        bigram_2 = relevant[deep][1]
        trigram_3=relevant[deep][2]
        sentence.append(trigram_3)
        while(1):
            relevant = []
            for i in quadgram_count:
                if i[0]==start and i[1] ==bigram_2 and i[2]==trigram_3:
                    relevant.append(i)
            prob = []
            for i in relevant:
                prob.append(mle_quadgram[i])
            arr = np.random.multinomial(1, prob,size = None)
            max_ = {0:0}
            for i in arr:
                max_[0] = max(max_[0],i)
            for i in range(len(arr)):
                if arr[i] == max_[0]:
                    deep = i
                    break
            #print(relevant)
            start = relevant[deep][1]
            bigram_2=relevant[deep][2]
            trigram_3 = relevant[deep][3]
            #print(trigram_3)
            sentence.append(trigram_3)
            relevant=[]
            deep = -100
            if trigram_3=='</s>':
                break
        return ' '.join(sentence)

In [12]:
print(' This is the sentence generated using unigram \n\n')
for i in range(10):
    print(generate(1))
print()
print(' This is the sentence generated using bigram \n\n')
for i in range(10):
    print(generate(2))
print()
print(' This is the sentence generated using trigram \n\n')
for i in range(10):
    print(generate(3))
print()
print(' This is the sentence generated using quadgram \n\n')
for i in range(10):
    print(generate(4))
print()

 This is the sentence generated using unigram 


<s> felt ’‘But silence:at a Oh, and changed, and Alice, now stupidly into send humbly: fact here? ‘to world a <s> said said </s>
<s> <s> succeeded you, a long flustered song, Oh, is!’‘Why me! again of Quadrille she had ’‘Nobody IS putthe Alice; they <s> tea,’ than talking the would a round, ‘and <s> one sneezing, the out passion, thought end </s>
<s> of bythe ‘or that (We youth, Alice perfectlyround, and room theothers threwthemselves could yourfinger the she bebeheaded, the out upon ’‘I back she bowed, and to </s>
<s> end! ‘I they vanished began everymoment can’t you’ll with the had the ‘But the exclaimed the THE The stuff? </s>
<s> saying said wenton theLory Cheshire over refused </s>
<s> asleep white all under moment at nurse Lory VIII <s> pace,’ far inher the could, OF little her, you said the depends she the remark checked dreadfullypuzzled it story,’ some good said first to pig, </s>
<s> say, all her, sleepy, a can to </s>
<s> </s>

## Q5., Q6. and Q7. follow below

##  Add One Smoothing and Good Turing Smoothing

### Below cell finds all the possible bigrams and puts all of them in the bigram_total dictionary

In [27]:
bigram_total = {}
for i in unigram_count:
    for j in unigram_count:
        bigram_total[(i,j)] = 0
## Created above all the possible bigrams

### The cell below is used for performing Add-One Smoothing

In [28]:
add_one = {}
for data in bigram_total:
    if data not in bigram_count:
        add_one[data] = (1)/(unigram_count[data[0]]+ len(unigram_count))
    else:
        add_one[data] = (bigram_count[data]+1)/((unigram_count[data[0]]) + len(unigram_count))
del bigram_total

### Drasatic Changes in Counts


In [30]:
## The drastic changes after Add One Smoothing are
## Example 1
#print(mle_bigram[('pink', 'eyes')], add_one[('pink', 'eyes')])
print(bigram_count[('pink', 'eyes')], add_one[('pink', 'eyes')]*(bigram_count[('pink', 'eyes')]+1)*len(bigram_count)/(len(unigram_count) + len(bigram_count)))
## Example 2unigr
#print(mle_bigram[('book,’', 'thought')], add_one[('book,’', 'thought')])
print(bigram_count[('book,’', 'thought')], add_one[('book,’', 'thought')]*(bigram_count[('book,’', 'thought')]+1)*len(bigram_count)/(len(unigram_count) + len(bigram_count)))
## Example 3
#print(mle_bigram[('or', 'conversations')], add_one[('or', 'conversations')])
print(bigram_count[('or', 'conversations')], add_one[('or', 'conversations')]*(bigram_count[('or', 'conversations')]+1)*len(bigram_count)/(len(unigram_count) + len(bigram_count)))

1 0.000502186074736095
1 0.0005021016878977928
1 0.0004972553078182335


#### The above change in counts is due to the fact that Add-One has taking too much mass from something that occurs and has assigned it to something that has never occurred. 

### Perplexity for Add_One Smoothing on test data


In [16]:
import math
def add_one_perplexity(data):
    prob = 0
    count = 0
    for i in data:
        arr = i.split()
        for j in range(len(arr)):
            count+=1
            #print(math.log(add_one[(arr[j],arr[j+1])]))
            try:
               
                prob+=math.log(add_one[(arr[j],arr[j+1])])
                #print('b')
            except:
                continue
    return math.exp(-1/(count)*prob)
print(' The perplexity of Add One Smoothing is ',add_one_perplexity(test_data))
del add_one

 The perplexity of Add One Smoothing is  1324.7182741125455


### Good Turing Count and discounting value

In [17]:
## Computed the count of the number of bigrams with a particular count
good_turing = {}
for i in bigram_count:
    if bigram_count[i] in good_turing:
        good_turing[bigram_count[i]]+=1
    else:
        good_turing[bigram_count[i]]=1

### 1. The below plot is almost a straight line for value <5. The discounting is ~0.6 to 0.8 for c<7.  
Note that 0 has a very high count because of the fact that number of times one word occurs is the maximum. I have not taken till 10 because of missing N-c values. 

In [18]:
after_count = {}
after_count[0] = good_turing[1]/(sum(good_turing.values()))
for i in range(1,11):
    after_count[i] = (good_turing[i+1]*(i+1))/(good_turing[i])
import matplotlib.pyplot as plt
plt.xlabel('c value used for c+1 estimation')
plt.ylabel(' The discounting value obtained')
plt.scatter(after_count.keys(),after_count.values())
print(' The discounting values are: ', after_count)

 The discounting values are:  {0: 0.8486194136066041, 1: 0.189990607808936, 2: 1.0593220338983051, 3: 1.736, 4: 3.3640552995391704, 5: 3.452054794520548, 6: 5.25, 7: 4.444444444444445, 8: 9.0, 9: 6.571428571428571, 10: 9.08695652173913}


### If there is a missing N(c+1) or N(c), then I am using the bigram mle here. 
### Below is the computation of probabilities using Good Turing Smoothing on the dataset

In [19]:
gt = {}
sum_ = sum(bigram_count.values())
for data in bigram_total:
    if data not in bigram_count:
        gt[data] = good_turing[1]/(sum(good_turing.values()))
    else:
        try:
            gt[data] = (good_turing[bigram_count[data]]*bigram_count[data])/(good_turing[bigram_count[data]-1]*sum_)
        except:
            gt[data] = mle_bigram[data]

### Perplexity for Good Turing on test data

In [20]:
def good_turing_perplexity(data):
    prob = 0
    count = 0
    for i in data:
        arr = i.split()
        for j in range(len(arr)):
            count+=1
            #print(math.log(add_one[(arr[j],arr[j+1])]))
            try:
                prob+=math.log(gt[(arr[j],arr[j+1])])
                #print('b')
            except:
                continue
    return math.exp(-1/(count)*prob)
print (good_turing_perplexity(test_data))
del gt
del bigram_total

135.81947749997084


#### Clearly among both Add-One and Good Turing, Good Turing performs ~ 10 times better with a perplexity value of 135.82 over Add-One's Perplexity Value of 1324.72